In [2]:
import math
import pandas as pd
from fuzzywuzzy import fuzz

In [3]:
restaurant_data = pd.read_csv("./vibes.csv")
hygiene_data = pd.read_csv("./place_hygiene.csv")

In [4]:
def find_best_match(restaurant_row):
    # Filter hygiene data by matching postcode
    matching_hygiene = hygiene_data[
        hygiene_data["PostCode"] == restaurant_row["post_code"]
    ]

    best_match = None
    highest_match = 0

    for _, hygiene_row in matching_hygiene.iterrows():
        similarity = fuzz.partial_ratio(
            restaurant_row["business_name"], hygiene_row["BusinessName"]
        )
        if similarity > highest_match:
            highest_match = similarity
            best_match = hygiene_row
    if highest_match < 51:
        best_match = None
    # print(f"match for - {restaurant_row['business_name']}: {highest_match}")
    return best_match

In [5]:
for idx, row in restaurant_data.iterrows():
    try:
        if math.isnan(row["RatingValue"]):
            best_match = find_best_match(row)
            if best_match is not None:
                restaurant_data.at[idx, "RatingValue"] = best_match["RatingValue"]
                restaurant_data.at[idx, "compare_name"] = best_match["BusinessName"]
    except TypeError:
        continue

In [6]:
restaurant_data.to_csv("vibes_2.csv", index=False)